In [1]:
COLORS = {
    "Artery-Aorta":"salmon",
    "Artery-Tibial": "red",
    "Adipose-Subcutaneous": "darkorange",    
    "Adipose-Visceral(Omentum)":"orange",
    "Brain-Caudate(basalganglia)":"lemonchiffon"   , 
    "Brain-Cerebellum":"yellow",
    "Cells-Transformedfibroblasts": "skyblue",
    "Esophagus-Mucosa": "sienna",
    "Esophagus-Muscularis":"burlywood",
    "Heart-LeftVentricle":"darkviolet",
    "Lung": "greenyellow",
    "Muscle-Skeletal": "mediumslateblue",
    "Nerve-Tibial":"gold",
    "Skin-NotSunExposed(Suprapubic)":"blue",
    "Skin-SunExposed(Lowerleg)":"cornflowerblue",
    "Thyroid":"green",
    "WholeBlood": "m"
        }

SHORTEN = {
    "Artery-Aorta":"Artery A."     ,
    "Artery-Tibial": "Artery T.",
    "Adipose-Subcutaneous": "Adipose S.",    
    "Adipose-Visceral(Omentum)":"Adipose V.",
    "Brain-Caudate(basalganglia)":"Caudate"   , 
    "Brain-Cerebellum":"Cerebellum",
    "Cells-Transformedfibroblasts": "Fibroblast",
    "Esophagus-Mucosa": "E. Mucosa",
    "Esophagus-Muscularis":"E Muscularis",
    "Heart-LeftVentricle":"Ventricule",
    "Lung": "Lung",
    "Muscle-Skeletal": "Muscle",
    "Nerve-Tibial":"Nerve",
    "Skin-NotSunExposed(Suprapubic)": "Skin Unexposed",
    "Skin-SunExposed(Lowerleg)":"Skin Leg",
    "Thyroid":"Thyroid",
    "WholeBlood": "Blood"
}

import math
import numpy as np
import os
import pandas as pd
import random
import sys
from scipy import stats, integrate
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(color_codes=True)

tissu = sorted(COLORS.keys())
CHR = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,'X']
Gene_table = pd.read_csv('/storage/szfeupe/Runs/GTEx_estr/FEATURES/Genes_only_table', sep='\t')
Motifs = pd.read_csv('/storage/resources/dbase/human/hg19/hg19.hipstr_reference_withmotif.bed', sep='\t', header=None)
Motifs.columns = ['chrom','str.start','str.end','motif.len','motif']

In [2]:
A=[]
B=[]
All_eSTRs=[]
for Tissue in tissu: 
    
    CAUSE='/storage/szfeupe/Runs/650GTEx_estr/Analysis_by_Tissue/'+Tissue+'/HH/caviar.out'
    LR = '/storage/szfeupe/Runs/650GTEx_estr/Analysis_by_Tissue/'+Tissue+'/PQValues'
    ANOV = '/storage/szfeupe/Runs/650GTEx_estr/Analysis_by_Tissue/'+Tissue+'/HH/anova_wg.csv'
    #Anova + LR
    anova = pd.read_csv(ANOV, sep=',')
    anova = anova[['chrom','gene','str.start','anova_pval','estr_fdr','esnp_fdr','delta_bic','delta_aic']]
    lr = pd.read_csv(LR,sep='\t')
    Table = pd.merge(lr, anova, on=['chrom','gene','str.start'])
    #Causal +Anova +LR
    Cs=[]
    for i in CHR:
        C=pd.read_csv(CAUSE+str(i), sep='\t')
        Cs.append(C)
    causal=Cs[0]
    for i in range(1,23,1):
        causal=causal.append(Cs[i])   
    CA = causal
    CA.index = CA['gene']
    Table.index = Table['gene']
    Master = pd.merge(Table,CA, on=['chrom','gene'])
    #Gene names
    genes=Gene_table.loc[Gene_table['gene'].isin(list(Master['gene']))]
    genes=genes[['gene','chrom','gene.name']]
    Master=pd.merge(Master, genes, on=['chrom','gene'])
    #motifs
    Master=pd.merge(Master, Motifs [['chrom','str.start','motif','str.end']], on=['chrom','str.start'])
    A.append(Master.loc[Master['gene.name']=='REL'] )
    B.append(Master.loc[Master['gene.name']=='CSTB'] )
    Head=['chrom','gene','gene.name','str.id','str.start','best.str.start', 'significant','qvalue', 'best.str.score','top.variant','top.variant.score','anova_pval','delta_bic','delta_aic','p.wald', 'beta','NTEST','beta.se', 'estr_fdr','esnp_fdr', 'motif']
    #Master[Head].to_csv('/storage/szfeupe/Runs/650GTEx_estr/Analysis_by_Tissue/'+Tissue+'/Master.table', sep='\t',  index=None)
    print(Master.shape, Tissue)
A = pd.concat(A) ##
B = pd.concat(B) ##
B['Tissues']=tissu
B.index=B['Tissues']


B['Tissues']=tissu
B.index=B['Tissues']
print('END!')
#NOT IN OUR DATA 
##   UGT1A1    VLDLR     MIF    PCA3  MXI1

#IN OUR DATA BUT NOT ESTRS
    #ARHG---       TRIB1    ARID1B   REL  

#IN OUR DATA with ESTRS
    #  CSTB




##   RUN The following
##   python ~/projects/GTEX_eSTRs/gtex-estrs/Scripts/STRs-vs-SNPs_effects/merge_causality_scores.py /storage/szfeupe/Runs/GTEx_estr/Analysis_by_Tissue/ WholeBlood,Cells-Transformedfibroblasts,Muscle-Skeletal,Lung,Artery-Tibial,Adipose-Subcutaneous,Esophagus-Mucosa causality > Merged_Best_causality.Table
##   To merge n select best by tissue


(11944, 22) Adipose-Subcutaneous
(11480, 23) Adipose-Visceral(Omentum)
(11041, 23) Artery-Aorta
(11662, 23) Artery-Tibial
(10352, 23) Brain-Caudate(basalganglia)
(10022, 23) Brain-Cerebellum
(10836, 23) Cells-Transformedfibroblasts
(11570, 23) Esophagus-Mucosa
(11223, 23) Esophagus-Muscularis
(10966, 23) Heart-LeftVentricle
(12686, 23) Lung
(11714, 23) Muscle-Skeletal
(12182, 23) Nerve-Tibial
(11337, 23) Skin-NotSunExposed(Suprapubic)
(11673, 23) Skin-SunExposed(Lowerleg)
(12227, 23) Thyroid
(11572, 22) WholeBlood
END!


In [5]:
#A = pd.concat(A) ##UGT1A1 
#B = pd.concat(B) ##MIF
B['Tissues']=tissu
B.index=B['Tissues']
B[['chrom','gene','gene.name','best.str.start', 'significant','qvalue', 'best.str.score','top.variant','top.variant.score','anova_pval','delta_bic','delta_aic','motif']]

,chrom,gene,gene.name,best.str.start,significant,qvalue,best.str.score,top.variant,top.variant.score,anova_pval,delta_bic,delta_aic,motif
Tissues,,,,,,,,,,,,,
Adipose-Subcutaneous,chr21,ENSG00000160213.5,CSTB,45196326,1,0.026765,0.001256,SNP_45233500,0.098794,3.623376e-01,-4.727337,-1.158992,CCCCG
Adipose-Visceral(Omentum),chr21,ENSG00000160213.5,CSTB,45196326,1,0.009244,0.164337,STR_45196326,0.164337,9.073061e-03,1.756690,4.944076,CCCCG
Artery-Aorta,chr21,ENSG00000160213.5,CSTB,45196326,1,0.013575,0.143539,STR_45196326,0.143539,8.581903e-04,6.091580,9.322689,CCCCG
Artery-Tibial,chr21,ENSG00000160213.5,CSTB,45196326,1,0.002820,0.999982,STR_45196326,0.999982,2.076818e-07,21.715432,25.313854,CCCCG
Brain-Caudate(basalganglia),chr21,ENSG00000160213.5,CSTB,45170834,0,0.724641,0.006582,SNP_45176268,0.075583,2.722182e-01,-3.396894,-0.752503,AC
Brain-Cerebellum,chr21,ENSG00000160213.5,CSTB,45277167,0,1.000000,0.004736,SNP_45243026,0.079672,3.317642e-01,-3.629022,-1.023852,AAC
Cells-Transformedfibroblasts,chr21,ENSG00000160213.5,CSTB,45196326,1,0.003467,0.998661,STR_45196326,0.998661,1.427390e-06,18.250389,21.625667,CCCCG
Esophagus-Mucosa,chr21,ENSG00000160213.5,CSTB,45196326,0,0.911693,0.005293,SNP_45158689,0.026499,1.423925e-01,-3.326554,0.182834,CCCCG
Esophagus-Muscularis,chr21,ENSG00000160213.5,CSTB,45196326,1,0.004218,0.990433,STR_45196326,0.990433,4.112573e-05,11.787493,15.110503,CCCCG


In [25]:
','.join(tissu)

'Adipose-Subcutaneous,Adipose-Visceral(Omentum),Artery-Aorta,Artery-Tibial,Brain-Caudate(basalganglia),Brain-Cerebellum,Cells-Transformedfibroblasts,Esophagus-Mucosa,Esophagus-Muscularis,Heart-LeftVentricle,Lung,Muscle-Skeletal,Nerve-Tibial,Skin-NotSunExposed(Suprapubic),Skin-SunExposed(Lowerleg),Thyroid,WholeBlood'

In [ ]:
#Heat scatter plots showing the heritability of each eQTL 
#apportioned to the most significant STR in the cis window 
#x = axis h2_str y = the additive effect from the top 100 most significant SNVs 

Master['h2b']= Master["cis_snp_h2"] + Master["cis_str_h2"]
MM1 = Master.loc[Master['h2b']<=.5]
ESTR = Master.loc[Master['qvalue']<0.1]
#plt.scatter(ESTR['cis_str_h2'],ESTR['h2b'])
sns.jointplot(x="cis_str_h2", y="h2b", data=MM1, kind="kde", color="g")
plt.show()
print (Master.shape, ' After restricting to h2b<=1 ',MM1.shape)